First step: manually copy to text file contents of ff.net pages. <br><br>
```webtext2dict``` converts this raw text into ```dict``` with summary:genre key/value pairs:

In [170]:
def webtext2dict(text):
    from re import findall
    slistlite = findall(r"iews\n.*\nR", text)
    slist = []
    for x in slistlite:
        slist.append(x[5:len(x)-2])
        
    glistlite = findall(r"h - .*? - C|h - Ch", text)
    glist = []
    for x in glistlite:
        glist.append(x[4:len(x)-4])
    
    sumgenr_dict = {}
    for x in range(len(slist)):
        sumgenr_dict[slist[x]] = glist[x]
        
    keyremoveset = set()
    for key in sumgenr_dict:
        if len(sumgenr_dict[key]) > 30:
            keyremoveset.add(key)
    
    for x in keyremoveset:
        sumgenr_dict.pop(x, None)

    return sumgenr_dict

def dictclean(inputdict):
    #1st pass for Hurt/Comfort > Hurt_Comfort, s-f>s_f naming inconsistencies b/c vars dont support dashes
    #2nd pass to get rid of other slashes (k/j > [k, j])

    from re import sub
    
    hc_repl_list = []
    
    for key in inputdict:
        if 'Hurt/Comfort' in inputdict[key]:
            hc_repl_list.append(key)
            
    for x in hc_repl_list:
        inputdict[x] = sub('Hurt/Comfort', 'Hurt_Comfort', inputdict[x])
        
        
    sf_repl_list = []
    
    for key in inputdict:
        if 'Sci-Fi' in inputdict[key]:
            sf_repl_list.append(key)
            
    for x in sf_repl_list:
        inputdict[x] = sub('Sci-Fi', 'Sci_Fi', inputdict[x])
    
    #1st pass done
    
    for key in inputdict:
        inputdict[key] = inputdict[key].split('/')
        
    #2nd pass done 
    
def ffdict_to_df(dfdict):
    import pandas as pd
    df = pd.DataFrame()
    df = df.assign(Summary=0, Adventure=0, Angst=0, Crime=0, Drama=0, Family=0,\
                   Fantasy=0, Friendship=0, General=0, Horror=0, Humor=0,\
                   Hurt_Comfort=0, Mystery=0, Parody=0, Poetry=0, Romance=0,\
                   Sci_Fi=0, Spiritual=0, Supernatural=0, Suspense=0, Tragedy=0, Western=0) #works for ffnet.
    for x in dfdict:
        r2 = {}
        r2['Summary'] = x
        for values in dfdict[x]:
            if values != '':
                r2[values] = 1
        df = df.append(r2, ignore_index=True)
        
    df = df.fillna(0)
    return df

def ffcoll_to_df(filename):
    #basically just go on ffnet and control a control c everything into a txt file.
    #This works with multiple pages in one file as well.
    f = open(filename, "r")
    rawtext1 = f.read()
    f.close 
    
    r1 = webtext2dict(rawtext1)
    dictclean(r1)
    df = ffdict_to_df(r1)
    
    return df

Now packaged into one function:

In [171]:
df = ffcoll_to_df("rawwebtext-p21-inuy-ffnet") #replace filename as needed

Last page scraped: page 21 (https://www.fanfiction.net/anime/Inuyasha/?&srt=4&lan=1&r=103&p=21). Equivalent to number of data points.

Now, this dict must be reformatted to one-hot encoding (technically not one-hot, but you get what I mean).

In [180]:
df.tail() #next step: get more varied content plssssssssssssssssssssss
#also extend to ao3 that would be infinitely more interesting

df.sum(axis=0) #possible to drop classes with low representation?
#get more data from more varied sources

Summary         Something goes awry during a trip through the ...
Adventure                                                    86.0
Angst                                                        32.0
Crime                                                         1.0
Drama                                                       107.0
Family                                                       20.0
Fantasy                                                       9.0
Friendship                                                   16.0
General                                                       0.0
Horror                                                        2.0
Humor                                                       174.0
Hurt_Comfort                                                 10.0
Mystery                                                       3.0
Parody                                                        2.0
Poetry                                                        0.0
Romance   

<h1>That's it.</h1>

In [181]:
df.to_csv('ff_train_1.csv') #code in colab depends on there being unnamed:0 column and im too lazy to fix that

In [173]:
#list(df.columns[1:]) #tags

In [174]:
#pred_df = pd.read_csv('predictions-ffnet-1.csv').drop('Unnamed: 0', axis=1)
#pred_df.tail()